In [1]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 12.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from neo4j import GraphDatabase
import json
import time

# Function to create nodes and relationships in Neo4j
def create_knowledge_graph(uri, user, password, json_file, batch_size=10):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def create_graph(tx, user_data):
        # Create user nodes
        for user_id, user_info in user_data.items():
            prompt = user_info.get('prompt', '')
            tx.run("""
                MERGE (u:User {user_id: $user_id})
                SET u += {age: $age, gender: $gender, occupation: $occupation, zip_code: $zip_code, prompt: $prompt}
            """, user_id=user_id, age=user_info['age'], gender=user_info['gender'], occupation=user_info['occupation'], zip_code=user_info['zip_code'], prompt=prompt)

            # Create movie nodes and relationships
            for rating in user_info['ratings']:
                tx.run("""
                    MERGE (m:Movie {movie_id: $movie_id})
                    SET m += {title: $title, genres: $genres}
                """, movie_id=rating['movie_id'], title=rating['title'], genres=rating['genres'])
                
                tx.run("""
                    MATCH (u:User {user_id: $user_id}), (m:Movie {movie_id: $movie_id})
                    MERGE (u)-[r:RATED {rating: $rating, timestamp: $timestamp}]->(m)
                """, user_id=user_id, movie_id=rating['movie_id'], rating=rating['rating'], timestamp=rating['timestamp'])

    with driver.session() as session:
        # Load the JSON data
        with open(json_file, 'r') as file:
            user_data = json.load(file)

        user_ids = list(user_data.keys())
        total_users = len(user_ids)
        
        # Process in batches
        for i in range(0, total_users, batch_size):
            batch_user_ids = user_ids[i:i+batch_size]
            batch_data = {user_id: user_data[user_id] for user_id in batch_user_ids}
            
            start_time = time.time()
            session.write_transaction(create_graph, batch_data)
            end_time = time.time()
            
            print(f"Processed batch {i//batch_size + 1}/{(total_users + batch_size - 1) // batch_size} in {end_time - start_time:.2f} seconds")
    
    driver.close()
    
# Usage
uri = "neo4j+s://dfc754bb.databases.neo4j.io"  
user = "neo4j"  
password = "f1osAWQTvu76DMo3RTlR-cOvnejRwC4XID2RFoO_1io"  
json_file = "restructured_user_data.json" 

create_knowledge_graph(uri, user, password, json_file)


/tmp/ipykernel_15312/1127917478.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, batch_data)


Processed batch 1/604 in 127.53 seconds
Processed batch 2/604 in 138.62 seconds
Processed batch 3/604 in 166.49 seconds
Processed batch 4/604 in 168.13 seconds
Processed batch 5/604 in 165.17 seconds
Processed batch 6/604 in 180.64 seconds
Processed batch 7/604 in 112.80 seconds
Processed batch 8/604 in 94.41 seconds
Processed batch 9/604 in 84.48 seconds
Processed batch 10/604 in 131.82 seconds
Processed batch 11/604 in 87.18 seconds
Processed batch 12/604 in 140.26 seconds
Processed batch 13/604 in 143.46 seconds
Processed batch 14/604 in 184.89 seconds
Processed batch 15/604 in 237.78 seconds
Processed batch 16/604 in 137.25 seconds
Processed batch 17/604 in 228.95 seconds
Processed batch 18/604 in 149.70 seconds
Processed batch 19/604 in 151.14 seconds
Processed batch 20/604 in 267.21 seconds
Processed batch 21/604 in 195.18 seconds
Processed batch 22/604 in 147.23 seconds
Processed batch 23/604 in 161.90 seconds
Processed batch 24/604 in 147.83 seconds
Processed batch 25/604 in 14

ClientError: {code: Neo.ClientError.Transaction.TransactionHookFailed} {message: You have exceeded the logical size limit of 400000 relationships in your database (attempt to add 1582 relationships would reach 400074 relationships). Please consider upgrading to the next tier.}